<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 200px; display: inline" alt="IMT"/> </a>
</center>

# [Ateliers: Technologies des grosses data](https://github.com/wikistat/Ateliers-Big-Data)

# Reconnaissance de caractères manuscrits ([MNIST](http://yann.lecun.com/exdb/mnist/)) avec <a href="https://cran.r-project.org/"><img src="https://cran.r-project.org/Rlogo.svg" style="max-width: 40px; display: inline" alt="R"/></a>

## 1 Introduction

### 1.1 Objetifs

Cet [atelier](https://github.com/wikistat/Ateliers-Big-Data/tree/master/MNIST) propose de comparer des versions R, Python et Spark. Ce calepin décline la solution en **R**.

Le site de Yann Le Cun: [MNIST DataBase](http://yann.lecun.com/exdb/mnist/), est la source des données étudiées, il  décrit précisément le problème et les modes d'acquisition. Il tient également à jour la liste des publications proposant des solutions avec la qualité de prévision obtenue. 

De façon très schématique, plusieurs stratégies sont développées dans une vaste littérature sur ces données.  
* Utiliser une méthode classique (k-nn, random forest...) sans trop raffiner mais avec des temps d'apprentissage rapide conduit à un taux d'erreur un peu supérieur à 3%.
* Ajouter  ou intégrer un pré-traitement des données permettant de recaler les images par des distorsions plus ou moins complexes.
* Construire une mesure de distance adaptée au problème, par exemple invariante par rotation, translation, puis l'intégrer dans une technique d'apprentissage classique. 
* Utiliser une méthode plus flexibles (réseau de neurones "profond", scattering) avec une optimisation fine des paramètres. 
* ...

L'**objectif** de ce calepin n'est pas de minimiser le taux d'erreur avec des méthodes sophistiquées mais d'utiliser ces données relativement volumineuses pour comparer diverses implémentations des méthodes d'apprentissage classiques. 

### 1.2 Lecture des données d'apprentissage et de test

In [ ]:
# Après avoir téléchargé les données
# Fichier train.csv
# Lecture des données 
Dtrain=read.csv("mnist_train.csv",header=F)
dim(Dtrain)

In [ ]:
Ltrain=as.factor(Dtrain[,785])
Dtrain=Dtrain[,-785]

In [ ]:
# Même chose avex les données de test

In [ ]:
Dtest=read.csv("mnist_test.csv",header=F)
Ltest=as.factor(Dtest[,785])
Dtest=Dtest[,-785]
dim(Dtest)

### 1.3 Exploration 

Les données ont déjà été normalisées centrées et ne présentent et sont complètes. Elles ne nécessitent pas d'autre "nettoyage" au moins rudimentaire.

Le [tutoriel](http://wikistat.fr/pdf/st-tutor3-python-scikit.pdf) d'introduction à Scikit-learn montre comment représenter les images des caractères ainsi qu'une ACP qui n'est pas reprise ici. 

On s'intéresse aux  performances de l'implémentation de k-means en R sur un tel volume.

In [ ]:
t1=Sys.time()
# Problème de convergence selon l'algorithme utilisé
clas.dig=kmeans(Dtrain,10, algorithm="Forgy",iter.max=50)
t2=Sys.time()
# temps d'exécution
difftime(t2,t1)

In [ ]:
classe=clas.dig$cluster
# Homogénéité des classes
table(Ltrain, classe)

## 2 Apprentissage et prévision du test

Tester ensuite différents modèles de discrimination. Bien entendu, il s'agirait d'optimiser les paramètres des modèles mais en prévoyant de restreindre la taille de l'échantillon d'apprentissage lorque les temps de calcul sont importants...

### 2.1 $K$ plus proches voisins

Les exécutions sont proposés sur un sous échantillon de l'échantillon d'apprentissage initial. Estimer le temps d'exécution sur tout l'échantillon ou l'obtenir ... de nuit!

La complexité de l'algorithme $k$-nn est en $O(nkd)$ où $d$ est la dimension, $k$, le nombre de voisins et $n$ la taille de l'échantillon d'apprentissage.

In [ ]:
# Sous échantillonnage
set.seed(11)
SousEch=sample(1:nrow(Dtrain),nrow(Dtrain)/5)
EchDtrain=Dtrain[SousEch,]
EchLtrain=Ltrain[SousEch]
dim(EchDtrain)

In [ ]:
library(class)
t1=Sys.time() 
prev.knn=knn(EchDtrain,Dtest,EchLtrain,k=10)
t2=Sys.time()
difftime(t2,t1)

In [ ]:
# matrice de confusion
conf=table(prev.knn,Ltest)
conf

In [ ]:
#taux d'erreur
(sum(conf)-sum(diag(conf)))/sum(conf)

### 2.2 Random Forest

L'algorithme random forest de la librairie `randomForest` est implémenté en fortran puis interfacé avec R. Il se montre plus efficace que celle de $k$-nn mais nettement plus long que l'implémentation de la librairie `ranger` par [Wright et Ziegler (2015)](http://arxiv.org/pdf/1508.04409v1.pdf).

#### `randomForest`

In [ ]:
## Random Forest avec sous-échantillon
library(randomForest)
t1=Sys.time() 
rf.fit=randomForest(x=EchDtrain,y=EchLtrain,xtest=Dtest,ytest=Ltest,ntree=100)
# mtry par défaut (sqrt(p))
t2=Sys.time()
difftime(t2,t1)

In [ ]:
rf.fit$test$err.rate[100,1]

#### `ranger`

Même calcul avec l'échantillon complet mais en utilisant l'implémentation de "ranger". Seul souci, la parallélisation (*multithreading*) n'est pas possible sous Windows alors qu'elle est automatique avec un autre système. Le temps d'apprentissage pourrait être encore sensiblement amélioré.

In [ ]:
library(ranger)
t1=Sys.time() 
DataF=data.frame("Ltrain"=Ltrain,Dtrain)
rf.fit=ranger(Ltrain~.,data=DataF,num.trees=100,write.forest=TRUE)
# mtry par défaut (sqrt(p)) 
t2=Sys.time()
difftime(t2,t1)

In [ ]:
predY=predict(rf.fit,dat=Dtest)
predY$class
conf=table(predY$predictions,Ltest)
erreur=(sum(as.vector(conf))-sum(diag(conf)))/nrow(Dtest)
print(erreur)

## 3 Effet de la taille de l'échantillon d'apprentissage avec `ranger`

La procédure d'estimation du modèle puis de prévision de l'échantillon test est itérée en fonction de la taille croissante de l'échantillon d'apprentissage.

In [ ]:
ntree=250
errMat=matrix(rep(0,36),nrow=12,ncol=3 )
dimnames(errMat)[[2]]=c("Taille","Temps","Erreur")
for (i in 1:12) {
  SousEch=sample(1:60000,5000*i)
  EchDtrain=Dtrain[SousEch,]
  EchLtrain=Ltrain[SousEch]
  n=dim(EchDtrain)
  t1=Sys.time() 
  DataF=data.frame("Ltrain"=EchLtrain,EchDtrain)
  rf.fit=ranger(Ltrain~.,data=DataF,num.trees=ntree,write.forest=TRUE)
  t2=Sys.time()
  errMat[i,1]=5000*i
  errMat[i,2]=difftime(t2,t1)
  predY=predict(rf.fit,dat=Dtest)
  conf=table(predY$predictions,Ltest)
  errMat[i,3]=(sum(as.vector(conf))-sum(diag(conf)))/nrow(Dtest)
}
print(errMat)

Résultats à mettre en forme:

      Taille     Temps Erreur
 [1,]   5000 13.888794 0.0545
 [2,]  10000 30.269731 0.0454
 [3,]  15000 47.969744 0.0419
 [4,]  20000  1.115497 0.0384
 [5,]  25000  1.506503 0.0359
 [6,]  30000  1.851889 0.0334
 [7,]  35000  2.154007 0.0341
 [8,]  40000  2.559163 0.0320
 [9,]  45000  2.997405 0.0320
[10,]  50000  3.335274 0.0298
[11,]  55000  3.595056 0.0287
[12,]  60000  4.010846 0.0283